# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter


if __name__ == "__main__":
    # Read the CSV file
    dataframe_log = pd.read_csv('../../data/logs/mobis.csv', sep=',')  

    # Drop the first column without knowing its name
    dataframe_log = dataframe_log.drop(dataframe_log.columns[0], axis=1)

    # Format the dataframe
    dataframe_log = pm4py.format_dataframe(
        dataframe_log, 
        case_id='case', 
        activity_key='activity', 
        timestamp_key='start'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_12235/1672979557.py:15: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe_log = pm4py.format_dataframe(


,activity,start,end,type,user,travel_start,travel_end,case,cost,case:concept:name,concept:name,time:timestamp,@@index,@@case_index
0,file travel request,2017-01-17 11:17:00+00:00,2017-01-17 11:23:00+00:00,Employee,JB8510,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,file travel request,2017-01-17 11:17:00+00:00,0,0
1,check if travel request needs preliminary pric...,2017-01-17 11:23:00+00:00,2017-01-17 11:24:00+00:00,Employee,JB8510,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,check if travel request needs preliminary pric...,2017-01-17 11:23:00+00:00,1,0
2,decide on approval requirements,2017-01-17 11:24:00+00:00,2017-01-17 11:24:00+00:00,Employee,JB8510,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,decide on approval requirements,2017-01-17 11:24:00+00:00,2,0
3,check if booking is necessary,2017-01-17 11:24:00+00:00,2017-01-17 11:40:00+00:00,Travel Department,KS9688,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,check if booking is necessary,2017-01-17 11:24:00+00:00,3,0
4,check if expense documents exist,2017-01-18 05:59:00+00:00,2017-01-18 06:31:00+00:00,Employee,JB8510,2017-10-01 00:00:00+00:00,2017-01-15 00:00:00+00:00,105,NaN,105,check if expense documents exist,2017-01-18 05:59:00+00:00,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55804,confirm travel expense report,2017-11-22 06:48:00+00:00,2017-11-22 06:50:00+00:00,Employee,KI9211,2017-11-19 00:00:00+00:00,2017-11-20 00:00:00+00:00,6348,NaN,6348,confirm travel expense report,2017-11-22 06:48:00+00:00,55804,3353
55805,decide on travel expense approval,2017-11-22 12:59:00+00:00,2017-11-22 13:06:00+00:00,Manager,AK7488,2017-11-19 00:00:00+00:00,2017-11-20 00:00:00+00:00,6348,NaN,6348,decide on travel expense approval,2017-11-22 12:59:00+00:00,55805,3353
55806,send original documents to archive,2017-11-29 20:12:00+00:00,2017-11-29 20:24:00+00:00,Employee,KI9211,2017-11-19 00:00:00+00:00,2017-11-20 00:00:00+00:00,6348,NaN,6348,send original documents to archive,2017-11-29 20:12:00+00:00,55806,3353
55807,calculate payments,2017-12-08 09:32:00+00:00,2017-12-08 09:55:00+00:00,Accounting,FQ3758,2017-11-19 00:00:00+00:00,2017-11-20 00:00:00+00:00,6348,NaN,6348,calculate payments,2017-12-08 09:32:00+00:00,55807,3353


# Drop unnessary columns

In [2]:
dataframe_log = dataframe_log.drop(columns=['case'])

In [3]:
dataframe_log = dataframe_log.drop(columns=['cost'])

In [4]:
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])

In [5]:
dataframe_log = dataframe_log.drop(columns=['concept:name'])

In [6]:
dataframe_log = dataframe_log.drop(columns=['@@index'])

In [7]:
dataframe_log = dataframe_log.drop(columns=['travel_start'])

In [8]:
dataframe_log = dataframe_log.drop(columns=['travel_end'])

In [9]:
dataframe_log = dataframe_log.drop(columns=['start'])

In [10]:
dataframe_log = dataframe_log.drop(columns=['end'])

In [11]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [12]:
dataframe_log

,activity,type,user,@@case_index,standardized_elapsed_time
0,file travel request,Employee,JB8510,0,-1.160944
1,check if travel request needs preliminary pric...,Employee,JB8510,0,-1.160857
2,decide on approval requirements,Employee,JB8510,0,-1.160842
3,check if booking is necessary,Travel Department,KS9688,0,-1.160842
4,check if expense documents exist,Employee,JB8510,0,-1.144594
...,...,...,...,...,...
55804,confirm travel expense report,Employee,KI9211,3353,-0.742182
55805,decide on travel expense approval,Manager,AK7488,3353,-0.736776
55806,send original documents to archive,Employee,KI9211,3353,-0.583572
55807,calculate payments,Accounting,FQ3758,3353,-0.404036


# Insert Start & End markers

In [13]:
# Define a function to insert start and end markers
def add_markers(df):
    # Identify unique case indices
    case_indices = df['@@case_index'].unique()
    
    # Prepare a container for new DataFrame rows
    new_rows = []
    
    # Iterate over each case index to add start and end markers
    for case_index in case_indices:
        # Create a start marker row with all columns except @@case_index set to 'Start'
        start_row = {col: 'Start' if col != '@@case_index' else case_index for col in df.columns}
        
        # Create an end marker row with all columns except @@case_index set to 'End'
        end_row = {col: 'End' if col != '@@case_index' else case_index for col in df.columns}
        
        # Append start row, rows for the current case, and end row
        new_rows.append(start_row)
        new_rows.extend(df[df['@@case_index'] == case_index].to_dict('records'))
        new_rows.append(end_row)
    
    # Convert the list of rows into a DataFrame
    return pd.DataFrame(new_rows)

# Apply the function to add start and end markers to the dataframe
modified_dataframe = add_markers(dataframe_log)

# Preprocess

In [14]:
modified_dataframe['standardized_elapsed_time'] = modified_dataframe['standardized_elapsed_time'].replace({'Start': 0, 'End': 1})

In [15]:
codes, uniques = pd.factorize(modified_dataframe['activity'])
modified_dataframe['activity'] = codes

In [16]:
# Fill NaN values with a placeholder before factorization
modified_dataframe['type'].fillna('missing', inplace=True)

# Factorize the 'type' column
codes, uniques = pd.factorize(modified_dataframe['type'])
modified_dataframe['type'] = codes

In [17]:
# Fill NaN values with a placeholder before factorization
modified_dataframe['user'].fillna('missing', inplace=True)

# Factorize the 'type' column
codes, uniques = pd.factorize(modified_dataframe['user'])
modified_dataframe['user'] = codes

In [18]:
modified_dataframe

,activity,type,user,@@case_index,standardized_elapsed_time
0,0,0,0,0,0.000000
1,1,1,1,0,-1.160944
2,2,1,1,0,-1.160857
3,3,1,1,0,-1.160842
4,4,2,2,0,-1.160842
...,...,...,...,...,...
62512,9,3,54,3353,-0.736776
62513,10,1,334,3353,-0.583572
62514,11,4,60,3353,-0.404036
62515,12,4,60,3353,-0.194961


### Padding for Cases with less then 5 events

In [19]:
# Calculating the frequency of each unique value in '@@case_index'
frequency = modified_dataframe['@@case_index'].value_counts()

# Finding the minimum occurrence
min_occurrence = frequency.min()

min_occurrence

13

In [20]:
""" # Assuming your dataframe might have different types, let's create a generic function to add rows
def add_rows(group, num_rows, case_index_value):
    # For each column, determine the appropriate "zero" value (int 0, string '', etc.)
    additional_rows = pd.DataFrame({
        column: 0 if pd.api.types.is_numeric_dtype(group[column]) else '' for column in group.columns
    }, index=range(num_rows))
    
    # Set the @@case_index column to the current case index value
    additional_rows['@@case_index'] = case_index_value
    
    # Append the additional rows to the group
    return pd.concat([group, additional_rows], ignore_index=True)

# Function to pad cases with less than 5 events
def pad_cases(df):
    # Group by @@case_index
    groups = df.groupby('@@case_index')
    
    # Placeholder for modified groups
    modified_groups = []
    
    for name, group in groups:
        # Calculate the number of events to add
        events_to_add = 5 - len(group)
        
        if events_to_add > 0:
            # Add the required number of rows
            group = add_rows(group, events_to_add, name)
        
        # Append the modified group to the list
        modified_groups.append(group)
    
    # Concatenate all modified groups back into a single DataFrame
    return pd.concat(modified_groups, ignore_index=True)  """

' # Assuming your dataframe might have different types, let\'s create a generic function to add rows\ndef add_rows(group, num_rows, case_index_value):\n    # For each column, determine the appropriate "zero" value (int 0, string \'\', etc.)\n    additional_rows = pd.DataFrame({\n        column: 0 if pd.api.types.is_numeric_dtype(group[column]) else \'\' for column in group.columns\n    }, index=range(num_rows))\n    \n    # Set the @@case_index column to the current case index value\n    additional_rows[\'@@case_index\'] = case_index_value\n    \n    # Append the additional rows to the group\n    return pd.concat([group, additional_rows], ignore_index=True)\n\n# Function to pad cases with less than 5 events\ndef pad_cases(df):\n    # Group by @@case_index\n    groups = df.groupby(\'@@case_index\')\n    \n    # Placeholder for modified groups\n    modified_groups = []\n    \n    for name, group in groups:\n        # Calculate the number of events to add\n        events_to_add = 5 - len(

In [21]:
""" # Apply the padding function
modified_dataframe = pad_cases(modified_dataframe)
modified_dataframe """

' # Apply the padding function\nmodified_dataframe = pad_cases(modified_dataframe)\nmodified_dataframe '

# Generate sliding windows

In [22]:
df_activity = modified_dataframe[['activity', '@@case_index']]
df_type = modified_dataframe[['type', '@@case_index']]
df_user = modified_dataframe[['user', '@@case_index']]
df_timestamp = modified_dataframe[['standardized_elapsed_time', '@@case_index']]

In [23]:
def generate_sliding_windows(df, case_id_column='@@case_index', window_size=5):
    windows = []
    targets = []
    case_indices = []

    # Iterate over each unique case
    for case_id in df[case_id_column].unique():
        # Extract the case
        case_data = df[df[case_id_column] == case_id]
        
        # Convert case_data to a NumPy array and drop the case_id_column
        case_data_array = case_data.drop(columns=[case_id_column]).to_numpy()

        # Adjusting the condition to correctly reflect window_size without needing an additional +1
        # Now it correctly considers window_size as including the target event
        if len(case_data_array) >= window_size:
            # Adjust the loop to generate sliding windows of size window_size - 1 for the inputs and use the next event as the target
            for i in range(len(case_data_array) - window_size + 1):
                # window now has window_size - 1 events
                window = case_data_array[i:i + window_size - 1]
                # The target is the event immediately following the window
                target = case_data_array[i + window_size - 1]
                windows.append(window)
                targets.append(target)
                case_indices.append(case_id)  # Store the case_id corresponding to the window

    # Convert lists to numpy arrays for easier handling and to ensure they are two-dimensional
    windows_array = np.array(windows)
    targets_array = np.array(targets)
    case_indices_array = np.array(case_indices)
    
    return windows_array, targets_array, case_indices_array

In [24]:
windows_activity, targets_activity, case_indices = generate_sliding_windows(df_activity)
windows_type, targets_type, case_indices = generate_sliding_windows(df_type)
windows_user, targets_user, case_indices = generate_sliding_windows(df_user)
windows_timestamp, targets_timestamp, case_indices = generate_sliding_windows(df_timestamp)

# LSTM

### Architecture

In [25]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Embedding

# Assuming these values as placeholders, replace them with actual counts from your data
num_users = modified_dataframe['user'].nunique()
num_activities = modified_dataframe['activity'].nunique()
num_types = modified_dataframe['type'].nunique()

embedding_dim_user = 50
embedding_dim_activity = 50
embedding_dim_type = 50

time_steps = 4

# Input layers
input_user = Input(shape=(time_steps,), name='input_user')
input_activity = Input(shape=(time_steps,), name='input_activity')
input_type = Input(shape=(time_steps,), name='input_type')
input_timestamp = Input(shape=(time_steps, 1), name='input_timestamp')

# Embedding layers
embedding_user = Embedding(input_dim=num_users, output_dim=embedding_dim_user, input_length=time_steps)(input_user)
embedding_activity = Embedding(input_dim=num_activities, output_dim=embedding_dim_activity, input_length=time_steps)(input_activity)
embedding_type = Embedding(input_dim=num_types, output_dim=embedding_dim_type, input_length=time_steps)(input_type)

# LSTM layers
lstm_user = LSTM(25, return_sequences=False)(embedding_user)
lstm_activity = LSTM(25, return_sequences=False)(embedding_activity)
lstm_type = LSTM(25, return_sequences=False)(embedding_type)
lstm_timestamp = LSTM(25, return_sequences=False)(input_timestamp)

# Concatenate outputs
concatenated = Concatenate(axis=-1)([lstm_user, lstm_activity, lstm_type, lstm_timestamp])

# Output layers
output_user = Dense(num_users, activation='softmax', name='output_user')(concatenated)
output_activity = Dense(num_activities, activation='softmax', name='output_activity')(concatenated)
output_type = Dense(num_types, activation='softmax', name='output_type')(concatenated)
output_timestamp = Dense(1, activation='linear', name='output_timestamp')(concatenated)

# Create and compile the model
model = Model(inputs=[input_user, input_activity, input_type, input_timestamp], 
              outputs=[output_user, output_activity, output_type, output_timestamp])

model.compile(optimizer='adam', 
              loss={'output_user': 'categorical_crossentropy', 
                    'output_activity': 'categorical_crossentropy', 
                    'output_type': 'categorical_crossentropy', 
                    'output_timestamp': 'mean_squared_error'},
              metrics={'output_user': 'accuracy', 
                       'output_activity': 'accuracy', 
                       'output_type': 'accuracy', 
                       'output_timestamp': 'mean_absolute_error'})

model.summary()

2024-08-19 22:12:39.444306: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_user (InputLayer)     [(None, 4)]                  0         []                            
                                                                                                  
 input_activity (InputLayer  [(None, 4)]                  0         []                            
 )                                                                                                
                                                                                                  
 input_type (InputLayer)     [(None, 4)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 4, 50)                16800     ['input_user[0][0]']      

# Data Splitting

In [26]:
from sklearn.model_selection import train_test_split

# Assuming the data is stored in variables named windows_user, windows_activity, windows_type, windows_timestamp
# and their corresponding targets are targets_user, targets_activity, targets_type, targets_timestamp

# Split data for user
train_user, test_user, train_targets_user, test_targets_user = train_test_split(
    windows_user, targets_user, test_size=0.3, random_state=42)

# Split data for activity
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

# Split data for type
train_type, test_type, train_targets_type, test_targets_type = train_test_split(
    windows_type, targets_type, test_size=0.3, random_state=42)

# Split data for timestamp
train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamp, targets_timestamp, test_size=0.3, random_state=42)

# Training

In [27]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def cyclic_lr(epoch, lr):
    # Example function that modulates LR within a range for each epoch
    # Customize this function based on your cyclic learning rate policy
    max_lr = 0.01  # Maximum LR
    base_lr = 0.001  # Base LR
    step_size = 10  # Number of epochs for half a cycle
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * np.maximum(0, (1 - x))
    return lr

lr_scheduler = LearningRateScheduler(cyclic_lr)

In [28]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Assuming num_users, num_activities, and num_types are defined as the number of unique categories for each attribute

# Converting user, activity, and type targets to categorical format for softmax output
train_targets_user_cat = to_categorical(train_targets_user, num_classes=num_users)
test_targets_user_cat = to_categorical(test_targets_user, num_classes=num_users)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities)

train_targets_type_cat = to_categorical(train_targets_type, num_classes=num_types)
test_targets_type_cat = to_categorical(test_targets_type, num_classes=num_types)

# Assuming the timestamp targets don't need conversion as they are continuous

history = model.fit([train_user, train_activity, train_type, train_timestamp], 
                    [train_targets_user_cat, train_targets_activity_cat, train_targets_type_cat, train_targets_timestamp],
                    epochs=25,
                    batch_size=64,
                    validation_data=([test_user, test_activity, test_type, test_timestamp], 
                                     [test_targets_user_cat, test_targets_activity_cat, test_targets_type_cat, test_targets_timestamp]),
                    verbose=1,
                    callbacks=[lr_scheduler])  # Add other callbacks as needed

Epoch 1/25
538/538 [==============================] - 24s 16ms/step - loss: 6.3968 - output_user_loss: 4.3097 - output_activity_loss: 1.2757 - output_type_loss: 0.6374 - output_timestamp_loss: 0.1741 - output_user_accuracy: 0.1598 - output_activity_accuracy: 0.6539 - output_type_accuracy: 0.7628 - output_timestamp_mean_absolute_error: 0.2923 - val_loss: 4.4728 - val_output_user_loss: 3.7683 - val_output_activity_loss: 0.3875 - val_output_type_loss: 0.2302 - val_output_timestamp_loss: 0.0867 - val_output_user_accuracy: 0.2186 - val_output_activity_accuracy: 0.8847 - val_output_type_accuracy: 0.9148 - val_output_timestamp_mean_absolute_error: 0.2020 - lr: 0.0010
Epoch 2/25
538/538 [==============================] - 13s 25ms/step - loss: 3.8024 - output_user_loss: 3.1963 - output_activity_loss: 0.3175 - output_type_loss: 0.2079 - output_timestamp_loss: 0.0807 - output_user_accuracy: 0.3451 - output_activity_accuracy: 0.8794 - output_type_accuracy: 0.9081 - output_timestamp_mean_absolute_e

In [29]:
evaluation = model.evaluate(
    [test_user, test_activity, test_type, test_timestamp],
    [test_targets_user_cat, test_targets_activity_cat, test_targets_type_cat, test_targets_timestamp]
)

print(evaluation)

461/461 [==============================] - 2s 3ms/step - loss: 1.3790 - output_user_loss: 0.8753 - output_activity_loss: 0.2633 - output_type_loss: 0.1864 - output_timestamp_loss: 0.0540 - output_user_accuracy: 0.7515 - output_activity_accuracy: 0.8832 - output_type_accuracy: 0.9141 - output_timestamp_mean_absolute_error: 0.1348
[1.3790110349655151, 0.8752849102020264, 0.26331740617752075, 0.1864219307899475, 0.053987275809049606, 0.7515443563461304, 0.883171558380127, 0.9141266942024231, 0.13484716415405273]


# Anomaly Score Computation

In [30]:
# Generate predictions for all inputs
predictions = model.predict([windows_user, windows_activity, windows_type, windows_timestamp])

# Extract predictions for categorical attributes (softmax probabilities) and continuous attribute (regression)
predictions_user = predictions[0]
predictions_activity = predictions[1]
predictions_type = predictions[2]
predictions_timestamp = predictions[3]

1535/1535 [==============================] - 8s 4ms/step


In [31]:
# Step 2: Compute Anomaly Scores for categorical variables (Resources and Activities)

def compute_anomaly_scores(predictions, actuals):
    # For categorical predictions, convert actuals to one-hot for comparison
    actuals_one_hot = to_categorical(actuals, num_classes=predictions.shape[-1])
    
    max_predictions = np.max(predictions, axis=-1)
    actual_predictions = np.sum(predictions * actuals_one_hot, axis=-1)  # Extract the probability of the actual class
    
    anomaly_scores = (max_predictions - actual_predictions) / max_predictions
    
    return anomaly_scores

anomaly_scores_user = compute_anomaly_scores(predictions_user, targets_user)
anomaly_scores_activity = compute_anomaly_scores(predictions_activity, targets_activity)
anomaly_scores_type = compute_anomaly_scores(predictions_type, targets_type)

In [32]:
def compute_anomaly_scores_continuous(predictions, actuals, normalization_factor):
    """
    Compute anomaly scores for continuous attributes.
    
    Parameters:
    - predictions: numpy array of predicted values.
    - actuals: numpy array of actual values.
    - normalization_factor: normalization factor (e.g., standard deviation of the attribute).
    
    Returns:
    - numpy array of anomaly scores.
    """
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize the differences
    anomaly_scores = differences / normalization_factor
    
    return anomaly_scores

In [33]:
normalization_factor = np.std(targets_timestamp)       # Example normalization factor (standard deviation)
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamp, normalization_factor)

In [34]:
import numpy as np

def classify_cases(anomaly_scores_user, anomaly_scores_activity, anomaly_scores_type, anomaly_scores_timestamp, threshold=0.98):
    # Ensure all inputs are numpy arrays of the same shape
    anomaly_scores_user = np.array(anomaly_scores_user).flatten()
    anomaly_scores_activity = np.array(anomaly_scores_activity).flatten()
    anomaly_scores_type = np.array(anomaly_scores_type).flatten()
    anomaly_scores_timestamp = np.array(anomaly_scores_timestamp).flatten()

    # Check if all arrays have the same length
    if not (len(anomaly_scores_user) == len(anomaly_scores_activity) == len(anomaly_scores_type) == len(anomaly_scores_timestamp)):
        raise ValueError("All input anomaly scores must have the same length.")

    # Find the maximum anomaly score across all attributes for each case
    max_scores = np.maximum.reduce([anomaly_scores_user, anomaly_scores_activity, anomaly_scores_type, anomaly_scores_timestamp])

    # Classify cases as anomalous if the maximum anomaly score exceeds the threshold
    anomalous_cases = max_scores > threshold
    
    return anomalous_cases

# Now use the anomaly scores for user, activity, type, and timestamp in the classification
anomalous_cases = classify_cases(anomaly_scores_user, anomaly_scores_activity, anomaly_scores_type, anomaly_scores_timestamp)

# Mapping

### True: anomaly, False: no anomaly

In [35]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted'] = anomalous_cases
mapping

,case,predicted
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
49096,3353,False
49097,3353,False
49098,3353,False
49099,3353,False


In [36]:
case_prediction = mapping.groupby('case')['predicted'].any()
case_prediction

case
0       False
1        True
2       False
3       False
4       False
        ...  
3349    False
3350    False
3351    False
3352     True
3353     True
Name: predicted, Length: 3354, dtype: bool

# Ground Truth

In [37]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [38]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/MobisToBe.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/295 [00:00<?, ?it/s]

In [39]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [40]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,0,0
1,1,1
2,0,0
3,1,0
4,1,0
...,...,...
3349,0,0
3350,1,0
3351,0,0
3352,0,1


# Evaluation

In [41]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [42]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.58


In [43]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.31


In [44]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.53


In [45]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.78


In [46]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.54
